In [ ]:
# Install necessary packages
!pip install bokeh pandas

In [201]:
# Import packages
from math import pi
import pandas as pd
import bokeh
from bokeh.plotting import show, figure, output_notebook
from bokeh.models import (AnnularWedge, ColumnDataSource,
                          Legend, LegendItem, Plot, Range1d, HoverTool)
from bokeh.layouts import Column, Row

# output_notebook()

In [111]:
# Import csv
inFile = "report.csv"

inDataFrame = pd.read_csv(inFile, encoding='utf-16',index_col=False)
# Remove planned expenses
inDataFrame = inDataFrame[~inDataFrame['Date'].isna()]
# Rename uncategorized expenses
inDataFrame[' Category'].fillna('Untitled',inplace=True)
inDataFrame = inDataFrame.fillna(0)
# inDataFrame.head()

In [112]:
#convert colors
dec2HexColor = lambda x: f'#{hex(int(0xFFFFFFFF)+int(x)+1)[-6:].upper()}'
inDataFrame[' Account Color'] = inDataFrame[' Account Color'].map(dec2HexColor)
inDataFrame[' To Account Color'] = inDataFrame[' To Account Color'].map(dec2HexColor)
inDataFrame[' Category Color'] = inDataFrame[' Category Color'].map(dec2HexColor)

In [ ]:
# Get years
inDataFrame['Date'] = pd.to_datetime(inDataFrame['Date'])
years = inDataFrame['Date'].dt.year.unique()

In [199]:
def piePlot(title: str, df: pd.Series, colors: list[str], percAggregate: int = 20):

    aggregated = df[~(df[' Type']=="TRANSFER")].groupby([' Category'])[' Amount'].sum()
    aggregated.sort_values(key=abs, inplace=True, ascending=False)
    
    # Categories as a percentage of the total
    perc = aggregated/aggregated.sum()
    
    # select the most important elements
    if len(aggregated) == 1:
        selected = aggregated
    else:
        selected = aggregated[perc.cumsum() < (1-percAggregate/100)]
        # at least one
        if len(selected) < 1:
            selected = aggregated.iloc[0]
        selected.loc["Other"] = aggregated.iloc[len(selected):].sum()

    # also aggregate perc
    perc = selected/selected.sum()
    
    angles = (2*pi*perc.cumsum()).tolist()

    source = selected.reset_index(name='value').rename(columns={' Category': "category"})
    source['start']  = [0] + angles[:-1]
    source['end'] = angles
    source['colors'] = [colors[cat] for cat in selected.index.to_list()]
    
    p = figure(height=350, title=title, toolbar_location=None,
           tools="hover", tooltips="@category: @value{0,0}", x_range=(-0.5, 1.0))

    p.wedge(x=0, y=1, radius=0.4,
            start_angle="start", end_angle="end",
            line_color="white", fill_color="colors", legend_field="category", source=source)
    
    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
    
    return(p)

def yearlyPlot(yr: int, df: pd.DataFrame):
    # Get category colors
    tmp = df[[' Category',' Category Color']]
    flt = df[' Category'].drop_duplicates().index
    tmp = tmp.loc[flt].to_dict(orient='split')
    colors = dict(tmp['data'])
    colors["Other"] = "#1C1C3C"

    dfN = df[df[' Amount'] < 0]
    dfP = df[df[' Amount'] >= 0]

    pN = piePlot(f"Yearly expenses by category ({yr})", dfN, colors, 15)
    pP = piePlot(f"Yearly income by category ({yr})", dfP, colors, 15)
    
    print(yr)
    # print(df.head())
    return [pN, pP]

In [226]:
def monthlyPlot(yr: int, df: pd.DataFrame):
    source = df.groupby([' Category', df.Date.dt.month])[' Amount'].sum()
    
    # Get category colors
    tmp = df[[' Category',' Category Color']]
    flt = df[' Category'].drop_duplicates().index
    tmp = tmp.loc[flt].to_dict(orient='split')
    colors = dict(tmp['data'])
    
    # create a new plot with a title and axis labels
    p = figure(title=f"Monthly expenses by category ({yr})",
               x_axis_label="Date", y_axis_label="Expenses",
              height=700, width=900)
    
    p.add_layout(Legend(), 'right')
    # add multiple renderers
    for cat in df[' Category'].unique():
        x = source[cat].index.to_list()
        y = source[cat].to_list()
        l = p.line(x, y, name=cat, legend_label=cat, color=colors[cat], line_width=2)
    p.add_tools(HoverTool(tooltips=[("Category", "$name"), ("Month", "@x{0}"), ("Expense", "@y{0,0}")],
                          mode="vline",
                         point_policy='snap_to_data'))#, renderers=[l]))

    p.legend.click_policy="hide"
    # show the results
    print(p)
    return(p)

In [ ]:
# Generate yearly reports
for yr in years:
    try:
        [pN, pP] = yearlyPlot(yr, inDataFrame[inDataFrame.Date.dt.year==yr])
        p = monthlyPlot(yr, inDataFrame[inDataFrame.Date.dt.year==yr])
        print(type(p))
        print(type(pN))
        print(type(pP))
        show(Column(Row(pN,pP),p))
    except Exception as e:
        print(yr,e)
        pass